# Clustering de perfiles de OkCupid
Este notebook muestra el proceso de análisis, preprocesamiento y entrenamiento de modelos para agrupar perfiles similares de la web de citas OkCupid.

## 1. Setup

Librerías importadas en el proyecto.

In [1]:
import numpy as np
import pandas as pd

import os

from IPython.display import IFrame

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

Variables globales declaradas en el proyecto.

In [2]:
RANDOM_SEED = 1337
np.random.seed(RANDOM_SEED)

#!git clone https://github.com/dietrujillo/ml_lab.git ./ml_lab 
DATA_PATH = os.path.join(os.pardir, "data")#os.path.join(os.curdir, "ml_lab", "lab3", "data")  # Path to csv data
INFO_PATH = os.path.join(os.pardir, "info")#os.path.join(os.curdir, "ml_lab", "lab3", "info")  # Path to problem info
RESULTS_PATH = os.path.join(os.pardir, "results")#os.path.join(os.curdir, "ml_lab", "lab3", "results")  # Path to results files

Enunciado del problema.

In [3]:
# Jupyter Notebook only
IFrame(os.path.join(INFO_PATH, "lab3.pdf"), width=1080, height=920)

## 2. Análisis exploratorio de los datos